# KINTSUGI: Knowledge Integration with New Technologies for Simplified User-Guided Image processing

### In the following notebook you will use the parameters found in A_Evaluation&Testing.ipynb to correct and stitch entire datasets.

## 1. Import packages. 
### *This must be done every time the notebook is started or restarted.

In [2]:
from concurrent.futures import ThreadPoolExecutor
import gc
import os as os
from tqdm.notebook import tqdm
import pandas as pd
import KCorrect
from Kstitch.stitching import stitch_images
from glob import glob
from skimage.io.collection import alphanumeric_key
import numpy as np
from skimage.io import imread_collection, imsave
import stackview
import numpy as np
from itertools import chain, repeat
import subprocess
from datetime import datetime
import imagej, scyjava
import logging
import warnings
import platform
current_dateTime = datetime.now()

## 2. Define directory paths. 
### *This must be done every time the notebook is started or restarted.

In [ ]:
base_dir = "C:/Users/smith6jt/"

In [3]:
image_dir = os.path.join(base_dir, 'KINTSUGI', 'data', '1904_CC2B_raw')
stitch_dir = image_dir.replace("_raw", "_BaSiC_Stitched")
meta_dir = stitch_dir.replace("_BaSiC_Stitched", "_meta")
project_file = os.path.join(meta_dir, "project_data.txt")
print(f"Image folder is {image_dir}.")
print(f"Stitching folder is {stitch_dir}.")
print(f"Meta folder is {meta_dir}.")

Image folder is /home/smith6jt/KINTSUGI/data/1904_CC2B_raw.
Stitching folder is /home/smith6jt/KINTSUGI/data/1904_CC2B_BaSiC_Stitched.
Meta folder is /home/smith6jt/KINTSUGI/data/1904_CC2B_meta.


## 3. Stitching and Illumination Correction

### 3.1 Stitching Function

The following cell defines the function that is called from the apply_basic function to stitch the corrected tiles.  It creates the model from the middle z-plane images of the first channel, and then applies the model to the rest of the images for all channels in the cycle.

In [7]:
def stitch(images_transformed, zplanes, dest, dest_1, channels, zplanes_n, pou, rows, cols, overlap_percentage, use_gpu):
    
    z = str(zplanes)
    ch = str(channels)
    pkl_dest = os.path.join(dest, "result_df.pkl")
    
    if zplanes == zplanes_n//2 and channels == 1:
        print(f"Start Stitching Z0{z.zfill(2)}_CH{ch}")
        if os.path.exists(pkl_dest):
            result_df = pd.read_pickle(pkl_dest)
        else:
            result_df, _ = stitch_images(images_transformed, rows, cols, initial_ncc_threshold=0.0, overlap_percentage=overlap_percentage, pou=pou, use_gpu)
            result_df.to_pickle(pkl_dest)
           
        
    else:
        print(f"Start Stitching Z0{z.zfill(2)}_CH{ch}")
        if os.path.exists(os.path.join(dest_1, "result_df.pkl")):
            result_df = pd.read_pickle(os.path.join(dest_1, "result_df.pkl"))

        else:
            print("Run registration channel to produce a stitching model.")

    result_df["y_pos2"] = result_df["y_pos"] - result_df["y_pos"].min()
    result_df["x_pos2"] = result_df["x_pos"] - result_df["x_pos"].min()
    
    size_y = images_transformed.shape[1]
    size_x = images_transformed.shape[2]
    
    stitched_image_size = (
        result_df["y_pos2"].max() + size_y,
        result_df["x_pos2"].max() + size_x,
    )
    stitched_image = np.zeros_like(images_transformed, shape=stitched_image_size)
    
    for i, row in result_df.iterrows():
        stitched_image[
            row["y_pos2"] : row["y_pos2"] + size_y,
            row["x_pos2"] : row["x_pos2"] + size_x,
        ] = images_transformed[i]

    result_image_file_path = os.path.join(dest,f"{z.zfill(2)}.tif") 
    imsave(result_image_file_path, stitched_image, check_contrast=False)
    
    print(f"Saved to {result_image_file_path}")


### 3.2 Illumination Correction Function

The following cell contains the function that will apply the BaSiC correction to your data.  Make sure the number of leading zero digits in your cycle folder names is correct for 'zeros' and the filename pattern is correct for 'im_raw'.  This function runs BaSiC for the middle z-plane images and then reuses the correction model for the rest of the channel's images.  It then calls the stitching function.

In [8]:
def apply_KCorrect(image_dir, stitch_dir, zplanes, cycles, channels, zplanes_n, pou, rows, cols, overlap_percentage, use_gpu):

    if_darkfield = True
    max_iterations = 500
    optimization_tolerance = 1e-6
    max_reweight_iterations = 25
    reweight_tolerance = 1.0e-3

    zeros = 3
    filename_pattern = f'1_000??_Z0{str(zplanes).zfill(2)}_CH{str(channels)}.tif'
    
    dest = os.path.join(stitch_dir, f"cyc{str(cycles).zfill(2)}", f"CH{str(channels)}")
    os.makedirs(dest, exist_ok=True)
    dest_1 = os.path.join(stitch_dir, f"cyc{str(cycles).zfill(2)}", "CH1")
    
    im_raw = sorted(glob(os.path.join(image_dir, f'cyc{str(cycles).zfill(zeros)}', filename_pattern)), key=alphanumeric_key)
    im = imread_collection(im_raw)
    im_array_init = np.asarray(im)
    dtype_max = np.iinfo(im_array_init.dtype).max
    im_array = im_array_init.astype(np.float64) / dtype_max

    if not np.all(np.isfinite(im_array)):
        raise ValueError("Input array contains inf or nan values")
    if np.any(im_array < 0):
        raise ValueError("Input array contains negative values")

    print(f"Start Illumination Correction cyc{str(cycles).zfill(2)} Z0{str(zplanes).zfill(2)}_CH{str(channels)}")
    flatfield, darkfield = KCorrect.KCorrect(im_array, if_darkfield = if_darkfield, max_iterations = max_iterations, optimization_tolerance = optimization_tolerance,  max_reweight_iterations = max_reweight_iterations, reweight_tolerance = reweight_tolerance)
    
    if np.any(np.isnan(flatfield)) or np.any(np.isnan(darkfield)):
        raise ValueError("Invalid flatfield or darkfield correction")
    if np.any(flatfield == 0):
        warnings.warn("Flatfield contains zero values which may cause division issues")
    
    corrected = np.zeros_like(im_array, dtype=np.float64)
    for i in range(len(im_array)):
        corrected[i] = ((im_array[i] - darkfield) / flatfield)
        corrected[i] = np.clip(corrected[i], 0, 1)
        
    KCorrect.validate_correction(im_array_init, corrected)
    corrected = (corrected * dtype_max).astype(np.uint16)                                       
    stitch(corrected, zplanes, dest, dest_1, channels, zplanes_n, pou, rows, cols, overlap_percentage, use_gpu)


### 3.3 Running Multiple Cycle/Channel Combinations

This cell runs the above two functions for all the images you define with start_cycle, end_cycle, start_channel, and end_channel.  You must enter these values as well as n, m, pou (determined from the first notebook), zplanes_n (number of zplanes), and overlap_percentage.  You must also enter the number of workers for the multithreading (usually determined by dataset size, available memory, and number of cores).  See https://docs.python.org/3/library/concurrent.futures.html

To run just one cycle or channel, simply enter the same number for both start and end.

If problems with the jupyter kernel restarting becomes an issue, decrease the number of workers, or close other programs to conserve resources.  You can also try reducing the histogram bins, number of iterations, and/or early_stop_n_iter_no_change values.

In [ ]:
n = 9  # Number of rows (height)
m = 7  # Number of columns (width)
start_cycle = 1
end_cycle = 1
start_channel = 2
end_channel = 2
pou = 13
zplanes_n = 17
overlap_percentage = 0.30
workers = zplanes_n-1
use_gpu = False

# Row coordinates: each row index is repeated m times
rows = list(chain.from_iterable(repeat(row, m) for row in range(n)))
# Column coordinates: snake pattern for each row, going back and forth from top left going right
cols = list(chain.from_iterable(range(m) if row % 2 == 0 else range(m - 1, -1, -1) for row in range(n)))

image_dir_list = [image_dir] * (zplanes_n-1)
stitch_dir_list = [stitch_dir] * (zplanes_n-1)
zplanes_list =[zplanes_n] * (zplanes_n-1)
pou_list = [pou] * (zplanes_n-1)
rows_list = [rows] * (zplanes_n-1)
cols_list = [cols] * (zplanes_n-1)
overlap_percentage_list = [overlap_percentage] * (zplanes_n-1)
use_gpu_list = [use_gpu] * (zplanes_n-1)

pbar_filesave = tqdm(total=100, unit="Percent",
                    bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]',
                    colour="green", position=0, leave=True)

for j in range(start_cycle, end_cycle+1):
    for i in range(start_channel, end_channel+1):

        cycles = j
        zplanes = zplanes_n//2 
        channels = i
        apply_KCorrect(image_dir, stitch_dir, zplanes, cycles, channels, zplanes_n, pou, rows, cols, overlap_percentage, use_gpu)
        
        if __name__ ==  '__main__':
            with ThreadPoolExecutor(max_workers=workers) as executor:
                cycles = [j] * (zplanes_n-1) 
                zplanes =  list(range(1, zplanes_n//2))+list(range((zplanes_n//2)+1, zplanes_n+1)) 
                channels = [i] * (zplanes_n-1)
                executor.map(apply_KCorrect, image_dir_list, stitch_dir_list, zplanes, cycles, channels, zplanes_list, pou_list, rows_list, cols_list, overlap_percentage_list, use_gpu_list) 

    gc.collect() 
    pbar_filesave.update(100 / (end_cycle - start_cycle + 1))

pbar_filesave.close()

## 4. Deconvolution

Before moving on to the deconvolution step, it is a good idea to review the results thus far.  If they are satisfactory, the parameters for deconvolution discovered in the first notebook can be entered below to define the decon function. Also, if more or all of the cycles have now been stitched, more deconvolution testing can be done on these stacks in the previous notebook before proceeding to a larger run. 

Be sure the MATLAB Runtime v9.5 (R2018, 64-bit) is installed.  Download for free here: http://www.mathworks.com/products/compiler/mcr/index.html. Reboot your computer after install.  

There is no need to supply a point-spread function.

The base_dir and stitch_dir definitions used previously will be again used here, and a new folder will be created to save the deconvolved images.  

The original deconvolution program was written specifically for lightsheet images. Here the estimation of the point-spread function was modified for use with widefield fluorescence images.  See https://www.nature.com/articles/s41598-019-53875-y for the original publication which includes links to the original code.

Output will be written to the terminal.  Check that the first image stack is processed without error.  If there is an "Maximum variable size allowed on the device is exceeded." error, then restart the kernel, decrease the max_GPU or max_CPU parameter, rerun the decon function cell, and try again.

In [5]:
def decon(base_dir, stitch_dir, cycles, channels):
    
    c = str(cycles)
    ch = str(channels)

    # pixel size in xy dimension (nanometers)
    xy_vox = 377
    # pixel size in z dimension (nanometers)
    z_vox = 1500
    # Number of iterations of Lucy-Richardson algo before stopping unless stop_crit is met first
    iterations = 100
    # Microscope objective numerical aperture
    mic_NA = 0.75
    # Refractive index of tissue being imaged
    tissue_RI = 1.3
    # Opening size in millimeters of objective aperture
    slit_aper = 6.5
    # Focal length in millimeters of objective
    f_cyl = 1
    # Used to reduce noise.  Increase value for noisy images. (0-10)
    damping = 0
    # If set, the deconvolved images will be clipped by this percent for max and min values, and then scaled to full range of bit depth. (0-5)
    hist_clip = 0.01
    # Percent change between iterations to use as criteria to stop deconvolution.
    stop_crit = 5.00
    # Enter 1 to perform on GPU, 0 to use CPU
    GPU = 1
    # Percent maximum GPU memory to use if GPU = 1
    max_GPU = 20
    # Percent maximum RAM to use if GPU = 0
    max_CPU = 40
    if GPU == 1:
        max_block=max_GPU
    elif GPU == 0:
        max_block=max_CPU

    # The respective excitation and emission wavelength in nanometers for each channel
    C1ex = 358
    C1em = 461
    C2ex = 753
    C2em = 775
    C3ex = 560
    C3em = 575
    C4ex = 648
    C4em = 668
    
decon_dir = stitch_dir.replace('_BaSiC_Stitched', '_Decon')
source = os.path.join(stitch_dir, f"cyc{str(dec_cycles).zfill(2)}", f"CH{str(dec_channels)}")
dest = os.path.join(decon_dir, f"cyc{str(dec_cycles).zfill(2)}", f"CH{str(dec_channels)}")
os.makedirs(dest, exist_ok=True)

os_system = platform.system()
if os_system == "Windows":

    decon_exe = os.path.join(base_dir, "LsDeconv.exe")
    print(f"Starting Deconvolution of cyc{c.zfill(2)} CH{ch}")
    if channels==1:
        subprocess.run([decon_exe, source, str(xy_vox), str(z_vox), str(iterations), str(mic_NA), str(tissue_RI), str(C1ex), str(C1em), str(f_cyl), str(slit_aper), str(damping), str(hist_clip), str(stop_crit), str(max_block), str(GPU)])
    if channels==2:
        subprocess.run([decon_exe, source, str(xy_vox), str(z_vox), str(iterations), str(mic_NA), str(tissue_RI), str(C2ex), str(C2em), str(f_cyl), str(slit_aper), str(damping), str(hist_clip), str(stop_crit), str(max_block), str(GPU)])
    if channels==3:
        subprocess.run([decon_exe, source, str(xy_vox), str(z_vox), str(iterations), str(mic_NA), str(tissue_RI), str(C3ex), str(C3em), str(f_cyl), str(slit_aper), str(damping), str(hist_clip), str(stop_crit), str(max_block), str(GPU)])
    if channels==4:
        subprocess.run([decon_exe, source, str(xy_vox), str(z_vox), str(iterations), str(mic_NA), str(tissue_RI), str(C4ex), str(C4em), str(f_cyl), str(slit_aper), str(damping), str(hist_clip), str(stop_crit), str(max_block), str(GPU)])

if os_system == "Linux":

    mat_dir = "/usr/local/MATLAB/MATLAB_Runtime/R2023b/"
    decon_exe = os.path.join("../K_Decon/for_redistribution_files_only/run_K_Decon.sh")
    print(f"Starting Deconvolution of cyc{c.zfill(2)} CH{ch}")
    if channels==1:
        subprocess.run([decon_exe, mat_dir, source, str(xy_vox), str(z_vox), str(iterations), str(mic_NA), str(tissue_RI), str(C1ex), str(C1em), str(f_cyl), str(slit_aper), str(damping), str(hist_clip), str(stop_crit), str(max_block), str(GPU)])
    if channels==2:
        subprocess.run([decon_exe, mat_dir, source, str(xy_vox), str(z_vox), str(iterations), str(mic_NA), str(tissue_RI), str(C2ex), str(C2em), str(f_cyl), str(slit_aper), str(damping), str(hist_clip), str(stop_crit), str(max_block), str(GPU)])
    if channels==3:
        subprocess.run([decon_exe, mat_dir, source, str(xy_vox), str(z_vox), str(iterations), str(mic_NA), str(tissue_RI), str(C3ex), str(C3em), str(f_cyl), str(slit_aper), str(damping), str(hist_clip), str(stop_crit), str(max_block), str(GPU)])
    if channels==4:
        subprocess.run([decon_exe, mat_dir, source, str(xy_vox), str(z_vox), str(iterations), str(mic_NA), str(tissue_RI), str(C4ex), str(C4em), str(f_cyl), str(slit_aper), str(damping), str(hist_clip), str(stop_crit), str(max_block), str(GPU)])

    try:
        os.rename(os.path.join(source, 'deconvolved'), os.path.join(dest, 'deconvolved'))
    except (FileNotFoundError):
        print("Reduce max memory.")

To apply the above decon function to multiple cycles/channels, enter the start and end numbers below.

In [ ]:
decon_start_cycle = 1
decon_end_cycle = 1
decon_start_channel = 2
decon_end_channel = 4

pbar_filesave = tqdm(total=100, unit="Percent",
                    bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]',
                    colour="green", position=0, leave=True)

for j in range(decon_start_cycle, decon_end_cycle+1):
    
    for i in range(decon_start_channel, decon_end_channel+1):
        
        decon(base_dir, stitch_dir, j, i)

    gc.collect() 
    pbar_filesave.update(100 / (decon_end_cycle - decon_start_cycle + 1))

pbar_filesave.close()

## 5. FIJI Clij2 plugin - EDF

In [3]:
channel_name_dict = {"cyc01.tif" : ["DAPI", "Blank1a", "Blank1b", "Blank1c"],
 "cyc02.tif" : ["DAPI", "CD31", "CD8", "Empty2c"],
 "cyc03.tif" : ["DAPI", "CD20", "Ki67", "CD3e"],
 "cyc04.tif" : ["DAPI", "SMActin", "Podoplanin", "CD68"],
 "cyc05.tif" : ["DAPI", "PanCK", "CD21", "CD4"],
 "cyc06.tif" : ["DAPI", "Lyve1", "CD45RO", "CD11c"],
 "cyc07.tif" : ["DAPI", "CD35", "ECAD", "CD107a"],
 "cyc08.tif" : ["DAPI", "CD34", "CD44", "HLADR"],
 "cyc09.tif" : ["DAPI", "Empty9a", "FoxP3", "CD163"],
 "cyc10.tif" : ["DAPI", "Empty10a", "CollagenIV", "Vimentin"],
 "cyc11.tif" : ["DAPI", "Empty11a", "CD15", "CD45"],
 "cyc12.tif" : ["DAPI", "Empty12a", "CD5", "CD1c"],
 "cyc13.tif" : ["DAPI", "Blank13a", "Blank13b", "Blank13c"]}

In [ ]:
ij_mem = 40
GPU_name = "NVIDIA RTX 4000 Ada Generation"
radius_x = 5
radius_y = 5
sigma = 20.0
edf_start_cycle = 1
edf_end_cycle = 13
edf_start_channel = 1
edf_end_channel = 4

os_system = platform.system()
if os_system == "Windows":
    javahome = os.path.join(base_dir, '/jdk-21_windows-x64_bin/jdk-21.0.5/bin/')
    os.environ['PATH'] = javahome + os.pathsep + os.environ['PATH']

scyjava.config.add_option(f'-Xmx{str(ij_mem)}g')
ij = imagej.init(os.path.join(base_dir, 'Fiji.app'))
decon_dir = stitch_dir.replace('_BaSiC_Stitched', '_Decon')
edf_dir = decon_dir.replace('_Decon', '_EDF')

macro = """
#@ File in_folder
#@ String device
#@ File out_folder
#@ String file_name
#@ Integer radius_x
#@ Integer radius_y
#@ Float sigma


File.openSequence(in_folder);
run("CLIJ2 Macro Extensions", "cl_device=[" + device + "]");
Ext.CLIJ2_clear();

image1 = "deconvolved";
newImage("extended_depth_of_focus_variance_projection573927482", "16-bit black", 9962, 9484, 1);
image2 = "extended_depth_of_focus_variance_projection573927482";

numTilesX = 1;
numTilesY = 2;
numTilesZ = 1;

tileWidth = 9962;
tileHeight = 4742;
tileDepth = 17;

for (x = 0; x < numTilesX; x++) {
	for (y = 0; y < numTilesY; y++) {
		for (z = 0; z < numTilesZ; z++) {

			Ext.CLIJ2_pushTile(image1, x, y, z, tileWidth, tileHeight, tileDepth, 0, 0, 0);
			
			Ext.CLIJ2_extendedDepthOfFocusVarianceProjection(image1, image2, radius_x, radius_y, sigma);
	
			Ext.CLIJ2_pullTile(image2, x, y, z, tileWidth, tileHeight, 1, 0, 0, 0);

		}
	}
}

Ext.CLIJ2_clear();
selectImage(image1);
close();
selectImage(image2);
saveAs("Tiff", out_folder + File.separator + file_name);
close();

"""
pbar_filesave = tqdm(total=100, unit="Percent",
                    bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]',
                    colour="green", position=0, leave=True)

for j in range(edf_start_cycle, edf_end_cycle+1):
    
    for i in range(edf_start_channel, edf_end_channel+1):

        edf_source = os.path.join(decon_dir, f"cyc{str(j).zfill(2)}", f"CH{str(i)}", "deconvolved")
        edf_dest = os.path.join(edf_dir, f"cyc{str(j).zfill(2)}")
        file_name = channel_name_dict.get(f"cyc{str(j).zfill(2)}.tif")[i-1]
        os.makedirs(edf_dest, exist_ok=True)

        args ={'in_folder': edf_source, "device" : GPU_name, "out_folder" : edf_dest, "file_name" : file_name, "radius_x" : radius_x, "radius_y" : radius_y, "sigma" : sigma}
        ij.py.run_macro(macro, args)

    gc.collect() 
    pbar_filesave.update(100 / (edf_end_cycle - edf_start_cycle + 1))

pbar_filesave.close()    

## 6. Registration

### 6.1 Combine channels for each cycle

In [ ]:
reg_start_cycle = 1
reg_end_cycle = 13
data_type = "uint16"  
pixel_size = float(0.377)

logging.basicConfig(level=logging.WARN) #change to INFO for more details
os_system = platform.system()
if os_system == "Windows":
    vipshome = os.path.join(base_dir, 'KINTSUGI','vips-dev-8.16','bin')
    os.environ['PATH'] = vipshome + ';' + os.environ['PATH']

import pyvips 

edf_dir = stitch_dir.replace('_BaSiC_Stitched', '_EDF')
reg_dir = edf_dir.replace('_EDF', '_Registration')
os.makedirs(reg_dir, exist_ok=True)

pbar_filesave = tqdm(total=100, unit="Percent",
                    bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]',
                    colour="green", position=0, leave=True)

for j in range(reg_start_cycle, reg_end_cycle + 1):

    cycle= f"cyc{str(j).zfill(2)}" 
     
    reg_source = os.path.join(edf_dir, f"cyc{str(j).zfill(2)}") 
    image_set = glob(os.path.join(reg_source, '*.tif'))
    channel_name_order = channel_name_dict.get(f"cyc{str(j).zfill(2)}.tif")
    image_set.sort(key=lambda x: channel_name_order.index(os.path.basename(x).split('.')[0]))
  
    channel_names = []
    for i in range(len(image_set)):
        split_name = os.path.basename(image_set[i].split(".")[0])
        channel_names.append(split_name)
 
    pyvips_image_set = [pyvips.Image.new_from_file(os.path.join(reg_source, filename), access="sequential") for filename in image_set]

    out_init = pyvips.Image.arrayjoin(pyvips_image_set, across=1)
    out = out_init.copy()
    out.set_type(pyvips.GValue.gint_type, "page-height", pyvips_image_set[0].height)

    x_dim = pyvips_image_set[0].width
    y_dim = pyvips_image_set[0].height

    bands = len(pyvips_image_set)
    outfile = os.path.join(reg_dir, f'cyc{str(j).zfill(2)}.ome.tif')
    out.set_type(pyvips.GValue.gstr_type, "image-description",
    f"""<?xml version="1.0" encoding="UTF-8"?>
        <OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
            xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
            xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
            <Image ID="Image:0" Name="{cycle}" >
                <!-- Minimum required fields about image dimensions -->
                <Pixels BigEndian="false"
                        DimensionOrder="XYCZT" 
                        ID="Pixels:0"
                        Interleaved="false"
                        SignificantBits="16"                                              
                        SizeC="{bands}" 
                        SizeT="1" 
                        SizeX="{x_dim}" 
                        SizeY="{y_dim}" 
                        SizeZ="1" 
                        Type="{data_type}" 
                        PhysicalSizeX="{pixel_size}" 
                        PhysicalSizeY="{pixel_size}">
                    <TiffData FirstC="0" FirstT="0" FirstZ="0" IFD="0" PlaneCount="1"/>
                    <Channel Color="16751615" ID="Channel:0:0" Name="{channel_names[0]}" IlluminationType="Epifluorescence" ContrastMethod="Fluorescence" AcquisitionMode="WideField" SamplesPerPixel="1"/>
                    <TiffData FirstC="1" FirstT="0" FirstZ="0" IFD="1" PlaneCount="1"/>
                    <Channel Color="7995391" ID="Channel:0:1" Name="{channel_names[1]}" IlluminationType="Epifluorescence" ContrastMethod="Fluorescence" AcquisitionMode="WideField" SamplesPerPixel="1"/>
                    <TiffData FirstC="2" FirstT="0" FirstZ="0" IFD="2" PlaneCount="1"/>
                    <Channel Color="9043967" ID="Channel:0:2" Name="{channel_names[2]}" IlluminationType="Epifluorescence" ContrastMethod="Fluorescence" AcquisitionMode="WideField" SamplesPerPixel="1"/>
                    <TiffData FirstC="3" FirstT="0" FirstZ="0" IFD="3" PlaneCount="1"/>
                    <Channel Color="1828651263" ID="Channel:0:3" Name="{channel_names[3]}" IlluminationType="Epifluorescence" ContrastMethod="Fluorescence" AcquisitionMode="WideField" SamplesPerPixel="1"/>
                </Pixels>
            </Image>
        </OME>""")

    out.tiffsave(outfile, subifd=True, page_height=y_dim, compression='lzw', tile=True,
                  tile_width=512, tile_height=512, pyramid=True, bigtiff=True)
    out_init = None
    out = None
    pyvips_image_set = None
    gc.collect() 
    pbar_filesave.update(100 / (reg_end_cycle - reg_start_cycle + 1))

pbar_filesave.close()

### 6.2 VALIS Registration

In [4]:
os_system = platform.system()
if os_system == "Windows":
    vipshome = os.path.join(base_dir, 'KINTSUGI','vips-dev-8.16','bin')
    os.environ['PATH'] = vipshome + os.pathsep + os.environ['PATH']

    javahome = os.path.join(base_dir, 'KINTSUGI','jdk-21_windows-x64_bin','bin')
    os.environ['PATH'] = javahome + os.pathsep + os.environ['PATH']

mavenhome = os.path.join(base_dir, 'KINTSUGI','apache-maven-3.9.9-bin','apache-maven-3.9.9', 'bin')
os.environ['PATH'] = mavenhome + os.pathsep + os.environ['PATH']

from valis import registration, slide_io
warnings.filterwarnings("ignore")

# registration.init_jvm()

edf_dir = stitch_dir.replace('_BaSiC_Stitched', '_EDF')
reg_dir = edf_dir.replace('_EDF', '_Registration')

slide_src_dir = reg_dir
results_dst_dir = os.path.join(reg_dir, "valis_out")
merged_slide_dst_f = reg_dir + "/merged.ome.tif"

In [ ]:
max_processed_image_dim_px = 600
registrar = registration.Valis(slide_src_dir, results_dst_dir, crop="overlap", max_processed_image_dim_px = max_processed_image_dim_px, imgs_ordered=True)
rigid_registrar, non_rigid_registrar, error_df = registrar.register()

In [ ]:
im_raw_deform = glob(os.path.join(reg_dir, "valis_out", os.path.basename(reg_dir), "deformation_fields", "*.png"))
im_deform = imread_collection(im_raw_deform)
deform = np.asarray(im_deform).astype(np.uint8)
stackview.slice(deform, continuous_update=True)

In [ ]:
max_non_rigid_registration_dim_px = 2500
registrar.register_micro(max_non_rigid_registration_dim_px=max_non_rigid_registration_dim_px)

In [ ]:
channel_name_dict = {"cyc01" : ["DAPI", "Blank1a", "Blank1b", "Blank1c"],
 "cyc02" : ["DAPI", "CD31", "CD8", "Empty2c"],
 "cyc03" : ["DAPI", "CD20", "Ki67", "CD3e"],
 "cyc04" : ["DAPI", "SMActin", "Podoplanin", "CD68"],
 "cyc05" : ["DAPI", "PanCK", "CD21", "CD4"],
 "cyc06" : ["DAPI", "Lyve1", "CD45RO", "CD11c"],
 "cyc07" : ["DAPI", "CD35", "ECAD", "CD107a"],
 "cyc08" : ["DAPI", "CD34", "CD44", "HLADR"],
 "cyc09" : ["DAPI", "Empty9a", "FoxP3", "CD163"],
 "cyc10" : ["DAPI", "Empty10a", "CollagenIV", "Vimentin"],
 "cyc11" : ["DAPI", "Empty11a", "CD15", "CD45"],
 "cyc12" : ["DAPI", "Empty12a", "CD5", "CD1c"],
 "cyc13" : ["DAPI", "Blank13a", "Blank13b", "Blank13c"]}

path_to_registrar = os.path.join(reg_dir, "valis_out", os.path.basename(reg_dir),"data", os.path.basename(reg_dir)+"_registrar.pickle")
registrar = registration.load_registrar(path_to_registrar)
merged_img, channel_names, ome_xml = registrar.warp_and_merge_slides(merged_slide_dst_f,
                                    crop=True,
                                    drop_duplicates=True,
                                    channel_name_dict=channel_name_dict,                                  
                                    tile_wh=1024,
                                    Q=90,
                                    pyramid=False
)
registration.kill_jvm()

In [ ]:
path_to_registrar = os.path.join(reg_dir, "valis_out", os.path.basename(reg_dir),"data", os.path.basename(reg_dir)+"_registrar.pickle")
registrar = registration.load_registrar(path_to_registrar)

for slide_name, slide_obj in registrar.slide_dict.items():
    reg_dir_out = os.path.join(reg_dir, "registered")
    os.makedirs(reg_dir_out, exist_ok=True)
    dst_f = os.path.join(reg_dir_out, f"{slide_name}.ome.tiff")
    slide_obj.warp_and_save_slide(dst_f=dst_f, crop="overlap", pyramid=False, interp_method = "bilinear", tile_wh=1024, Q=90)

registration.kill_jvm()

saving /home/smith6jt/KINTSUGI/data/1904_CC2B_Registration/registered/cyc01.ome.tiff (9953 x 9494 and 4 channels)

